In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import os

In [2]:
def load_image(path, max_size=512):
    image = Image.open(path).convert('RGB')
    size = min(max(image.size), max_size)
    transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor()
    ])
    return transform(image).unsqueeze(0)

def imshow(tensor, title=''):
    image = tensor.clone().squeeze(0).cpu().clamp(0, 1)
    plt.imshow(transforms.ToPILImage()(image))
    plt.axis('off')
    plt.title(title)
    plt.show()


In [3]:
def calc_mean_std(feat, eps=1e-5):
    N, C = feat.size()[:2]
    feat_var = feat.view(N, C, -1).var(dim=2) + eps
    feat_std = feat_var.sqrt().view(N, C, 1, 1)
    feat_mean = feat.view(N, C, -1).mean(dim=2).view(N, C, 1, 1)
    return feat_mean, feat_std

def adain(content_feat, style_feat):
    content_mean, content_std = calc_mean_std(content_feat)
    style_mean, style_std = calc_mean_std(style_feat)
    normalized = (content_feat - content_mean) / content_std
    return normalized * style_std + style_mean

class VGGEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        vgg = models.vgg19(pretrained=True).features
        self.encoder = nn.Sequential(*list(vgg.children())[:21])
        for param in self.encoder.parameters():
            param.requires_grad = False

    def forward(self, x):
        return self.encoder(x)

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(512, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, 3, 3, padding=1)
        )

    def forward(self, x):
        return self.model(x)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

content = load_image("resource\sty\content.png").to(device)
style = load_image("resource\sty\style.jpg").to(device)

encoder = VGGEncoder().to(device)
decoder = Decoder().to(device)

with torch.no_grad():
    content_feat = encoder(content)
    style_feat = encoder(style)
    t = adain(content_feat, style_feat)
    output = decoder(t)

imshow(content, "Content")
imshow(style, "Style")
imshow(output, "Stylized Output")

FileNotFoundError: [Errno 2] No such file or directory: 'resource\\sty\\content.png'